In [38]:
# Parse date taken from cr2 file header - note, this is from the camera clock
# Taken from https://stackoverflow.com/questions/3696642/reading-a-cr2-raw-canon-image-header-using-python

from struct import unpack_from

def FindDateTimeOffsetFromCR2( buffer, ifd_offset ):
    # Read the number of entries in IFD #0
    (num_of_entries,) = unpack_from('H', buffer, ifd_offset)

    # Work out where the date time is stored
    datetime_offset = -1
    for entry_num in range(0,num_of_entries-1):
        (tag_id, tag_type, num_of_value, value) = unpack_from('HHLL', buffer, ifd_offset+2+entry_num*12)
        if tag_id == 0x0132:
            return value

def get_cr2_timestamp(file):
    with open(file, "rb") as f:
            buffer = f.read(1024) # read the first 1kb of the file should be enough to find the date / time
            datetime_offset = FindDateTimeOffsetFromCR2(buffer, 0x10)
            return unpack_from('19s', buffer, datetime_offset)[0].decode()

In [66]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
from pathlib import Path

# Find coords of target using CDS name lookup
# Returns RA & DEC in degrees

def object_coords(obj_name):
    sc = SkyCoord.from_name(obj_name)
    return sc.ra.degree, sc.dec.degree

# Update headers of fits file with datetime, exposure, RA & DEC
# Used by Tycho for platesolving and NEO lookup

def update_fits_file_headers(file, dt, exp, ra, dec):
    with fits.open(file, mode='update') as hdul:
        hdr = hdul[0].header
        hdr['EXPOSURE'] = exp
        hdr['DATE-OBS'] = dt
        hdr['RA'] = ra
        hdr['DEC'] = dec

# Update all fits files in folder
# Datetime of image is taken from the equivalent cr2 file two folders above

def update_fits_files(dir, exp, obj_name):
    sc = SkyCoord.from_name(obj_name)
    ra = sc.ra.degree
    dec = sc.dec.degree
    fit_path = Path(dir)
    cr2_path = fit_path.parent.parent
    for p in fit_path.glob('*.fit'):
        cr2_name = p.with_suffix('.cr2').name
        cr2_file = cr2_path.joinpath(cr2_name)
        dt = get_cr2_timestamp(cr2_file)
        update_fits_file_headers(p, dt, exp, ra, dec)


In [64]:
EXP = 90.0
OBJ = 'M97'
DIR = 'C:/APT_Images/Camera_1/2022-03-19/M97/pipp_20231106_170337/L_M_97_0002_ISO800_90s__17C/'


In [67]:
update_fits_files(DIR, EXP, OBJ)